## Import Data


In [1]:
import pandas as pd
file = pd.read_csv("hdac.txt", sep = "\t")
### Get only IC50 that has equal acticit and the units of nM
query = file.query('STANDARD_TYPE == "IC50" and RELATION == "=" and STANDARD_UNITS == "nM"')
query.drop_duplicates(cols = "CANONICAL_SMILES")

IC50 = query['STANDARD_VALUE']
SMILES = query['CANONICAL_SMILES']


In [2]:
query.count(axis=0)

CMPD_CHEMBLID            707
MOLREGNO                 707
PARENT_CMPD_CHEMBLID     707
PARENT_MOLREGNO          707
MOL_PREF_NAME            131
COMPOUND_KEY             707
MOLWEIGHT                700
ALOGP                    697
PSA                      697
NUM_RO5_VIOLATIONS       697
CANONICAL_SMILES         707
ACTIVITY_ID              707
STANDARD_TYPE            707
RELATION                 707
STANDARD_VALUE           707
STANDARD_UNITS           707
PCHEMBL_VALUE            701
ACTIVITY_COMMENT           0
DATA_VALIDITY_COMMENT      6
POTENTIAL_DUPLICATE      707
BAO_ENDPOINT             707
UO_UNITS                 707
QUDT_UNITS               707
ASSAY_ID                 707
ASSAY_CHEMBLID           707
ASSAY_TYPE               707
DESCRIPTION              707
ASSAY_SRC_ID             707
ASSAY_SRC_DESCRIPTION    707
ASSAY_ORGANISM           567
ASSAY_STRAIN               0
ASSAY_TAX_ID             567
CURATED_BY               707
BAO_FORMAT               707
TID           

## Standardized molecules

In [3]:
from rdkit.Chem.SaltRemover import SaltRemover
from rdkit.Chem import MolFromSmiles,MolToSmiles

remover = SaltRemover()
len(remover.salts)

SMILES_desalt = []

for i in SMILES:
    mol = MolFromSmiles(i) 
    mol_desalt = remover.StripMol(mol)
    mol_SMILES = MolToSmiles(mol_desalt)
    SMILES_desalt.append(mol_SMILES)

## Binding the Standardized SMILES with Bioactivity Data

In [4]:
IC50_nm = pd.Series.tolist(IC50)

df = pd.DataFrame({'smiles' : SMILES_desalt,
 'IC50':IC50_nm
  })

## Pharsing the SMILES to calculate Morgan Fingerprint Descriptors

In [5]:
from rdkit.Chem import PandasTools, AllChem
PandasTools.AddMoleculeColumnToFrame(df, smilesCol = 'smiles')
fps = [AllChem.GetMorganFingerprintAsBitVect(m, 1) for m in df['ROMol']]

## Converting RDkit data structure to numerical descriptors

In [6]:
import numpy as np
from rdkit import DataStructs
descriptors = []
for fp in fps:
  arr = np.zeros((1,))
  DataStructs.ConvertToNumpyArray(fp, arr)
  descriptors.append(arr)


## Data Transformation (Converting IC50 to pIC50)

In [8]:
from math import log10
import numpy as np
np_ic50 = np.array(IC50)

pIC50 = []
for i in np_ic50:
    molar = i*(10**-9) # Converts nM to M
    pIC50.append(-log10(molar))


y = pIC50
X = descriptors

## Writing a function that will perform training, cross-validation and testing 

In [10]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.cross_validation import train_test_split, cross_val_predict, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np

def build_models_regression(x, y, model):
   
    
    if model == "training":
        X_train, X_test, y_train, y_test = train_test_split(x, y,
        test_size = 0.2)
        rf = RandomForestRegressor(n_estimators = 500, max_features = 'sqrt')
        rf.fit(X_train, y_train)
        rmse = np.sqrt(mean_squared_error(y_train, rf.predict(X_train)))
        r2 = r2_score(y_train, rf.predict(X_train))
        print("The internal set's R-Squared is %.2f  and Root Mean Squared Error is %.2f" % (r2, rmse))
    #rmse = np.sqrt(mean_squared_error(y_train, rf.predict(X_train)))
    #r2 = r2_score(y_train, rf.predict(X_train))
    #print "The internal set's R-Squared is %.2f  and Root Mean Squared Error is %.2f" % (r2, rmse)
    elif model == "CV":
        X_train, X_test, y_train, y_test = train_test_split(x, y,
        test_size = 0.2)
        rf = RandomForestRegressor(n_estimators = 500, max_features = 'sqrt')
        rf.fit(X_train, y_train)
        #rmse = np.sqrt(mean_squared_error(y_train, cross_val_predict(rf, X_train, y_train, cv = 10)))
        r2 = cross_val_score(rf, X_train, y_train, cv = 10)
        r2 = r2.mean()
        #r2 = r2_score(y_train, cross_val_predict(rf, X_train, y_train, cv = 10))
        print("The 10-fold Cross Validation set's R-Squared is %.2f" % r2)
    elif model == "testing":
        X_train, X_test, y_train, y_test = train_test_split(x, y,
        test_size = 0.2)
        rf = RandomForestRegressor(n_estimators = 500, max_features = 'sqrt')
        rf.fit(X_train, y_train)
        rmse = np.sqrt(mean_squared_error(y_test, rf.predict(X_test)))
        r2 = r2_score(y_test, rf.predict(X_test))
        print("The external set's R-Squared is %.2f  and Root Mean Squared Error is %.2f" % (r2, rmse))
    



## Training performance

In [11]:
build_models_regression(x = descriptors, y = pIC50, model = "training")

The internal set's R-Squared is 0.87  and Root Mean Squared Error is 0.39


## 10-Fold Cross Validation's Performance

In [12]:
build_models_regression(x = descriptors, y = pIC50, model = "CV")

The 10-fold Cross Validation set's R-Squared is 0.52


## Testing Performance

In [13]:
build_models_regression(x = descriptors, y = pIC50, model = "testing")

The external set's R-Squared is 0.51  and Root Mean Squared Error is 0.78
